# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
else:
    print("API key found and looks good so far!")


MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
openai = OpenAI()


system_message = "You are a helpful assistant"

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

OLLAMA_BASE_URL = 'http://localhost:11434'
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

def stream_ollama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = ollama.chat.completions.create(
        model=MODEL_LLAMA,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Llama":
        result = stream_ollama(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result
		

message_input = gr.Textbox(label="Your question:", info="Enter a question to be answered", lines=7)
message_output = gr.Markdown(label="Answer:")
model_selector = gr.Dropdown(["GPT", "Llama"], label="Select model", value="GPT")

view = gr.Interface(
    fn=stream_model,
    title="Question Answerer", 
    inputs=[message_input, model_selector], 
    outputs=[message_output], 
    examples=[
            ["Explain the Transformer architecture to a layperson", "GPT"],
            ["Explain the Transformer architecture to an aspiring AI engineer", "Claude"]
        ], 
    flagging_mode="never"
    )
view.launch()

/Users/usmanbabakolo/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/components/dropdown.py:230: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: Claude or set allow_custom_value=True.
  warnings.warn(


API key found and looks good so far!
* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
